<a href="https://colab.research.google.com/github/zhiyuan-95/Amex-default-prediction-/blob/main/prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=8ej3REfZLnkq6LMmx7fZPmSnNmnd7B&prompt=consent&access_type=offline&code_challenge=pKpg_Pg7j0puIB6PNjt52Lzul0MWocYe-PwSVaa33iU&code_challenge_method=S256

Enter authorization code: 

Command killed by keyboard interrupt

^C


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%shell
pip --quiet install kaggle

In [ ]:
!pip install keras-tuner -q
!pip3 install -U crcmod

In [ ]:
#!gsutil cp gs://aedp/expanded_data/train/part-00000 /content/train.csv
!gsutil cp gs://aedp/expanded_data/test/part-* /content/test.csv

In [ ]:
import csv
import pandas as pd
import numpy as np
import os
import sys
import tensorflow as tf
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow import keras
from keras_tuner.tuners import RandomSearch
import time
from keras import layers

In [ ]:
train = pd.read_csv('train.csv')
train.set_index('customer_ID', inplace = True)

In [ ]:
cat_top = ['B_30_top', 'B_38_top', 'D_114_top', 'D_116_top', 'D_117_top', 'D_120_top', 'D_126_top', 'D_63_top', 'D_64_top', 'D_66_top', 'D_68_top']
num = list(train.columns)
for x in cat_top: num.remove(x)

### Preprocessing

In [ ]:
def fill_na(data):
  data_cat = data[cat_top].astype(str).fillna('JIN')
  data_num = data[num].fillna(0)
  data_out = pd.concat([data_num,data_cat], axis = 1)
  return data_out

In [ ]:

from sklearn.preprocessing import StandardScaler
def stder(X):
    cols = X.columns
    stder = StandardScaler(copy=True, with_mean = True, with_std = True)
    stder.fit(X)
    return pd.DataFrame(stder.transform(X), columns = cols)

In [ ]:
encoder = OneHotEncoder(sparse_output = False)

In [ ]:
def encode_train(df,cat=cat_top):
  label = df[['target']]
  df = df.drop(columns = ['target'])
  #encode label to fit tensorflow
  encoded_label = encoder.fit_transform(label)
  encoded_label_names = encoder.get_feature_names_out()
  label_encoded = pd.DataFrame(encoded_label, columns=encoded_label_names)
  label_encoded.index = df.index
  #encode categorical data
  encoded_columns = encoder.fit_transform(df[cat])
  encoded_column_names = encoder.get_feature_names_out()
  encoded_df = pd.DataFrame(encoded_columns, columns=encoded_column_names, index = df.index)
  feature_encoded = pd.concat([df.drop(columns=cat), encoded_df], axis=1)
  return stder(feature_encoded.astype(np.float64)), label_encoded

In [ ]:
def preprocess_train(df):
  df_train = fill_na(df)
  X_train_encoded, y_train_encoded= encode_train(df_train)
  X_train, X_val, y_train, y_val = train_test_split(X_train_encoded, y_train_encoded, test_size=0.01, random_state=42)
  return X_train, X_val, y_train, y_val

In [ ]:
X_train, X_val, y_train, y_val= preprocess_train(train)

### Retrive best hyperparameter

In [ ]:
def model(hp):
    md = tf.keras.models.Sequential()
    md.add(layers.Flatten(input_shape=(1503,))),
    md.add(layers.BatchNormalization()),
    for i in range(n_layers):
      md.add(layers.Dense(hp.Int('input_units_{0}'.format(1+i),min_value = 200,max_value=3000,step = 50),activation=hp.Choice('activation_{0}'.format(i+1),['elu','sigmoid','tanh']))),
    md.add(layers.BatchNormalization()),

    md.add(layers.Dense(2, activation=hp.Choice('activation_end'.format(i),['sigmoid']))),
    lr = hp.Float("lr", min_value=1e-6, max_value=1e-5, sampling="log")

    md.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  metrics = ['accuracy'],
                  loss='mean_squared_error')
    return md

In [ ]:
record_path = '/content/drive/MyDrive/Colab Notebooks/Default prediction'
os.makedirs(record_path, exist_ok=True)

In [ ]:
n_layers = 2
epks = 15
tuner = RandomSearch(
    model,
    objective = "val_loss",
    max_trials =200,
    executions_per_trial = 3,
    directory = record_path,
    overwrite = False,
    project_name = 'model_{0}l'.format(n_layers)
    )

In [ ]:
best_hp = tuner.get_best_hyperparameters()[0].values

In [ ]:
best_hp

{'input_units_1': 3400,
 'activation_1': 'sigmoid',
 'input_units_2': 400,
 'activation_2': 'tanh',
 'activation_end': 'sigmoid',
 'lr': 9.220368359466073e-06}

### Predict

In [ ]:
def best_model(activation_end,n_layers=n_layers,**dense):
    md = tf.keras.models.Sequential()
    md.add(layers.Flatten(input_shape=(1503,))),
    md.add(layers.BatchNormalization()),
    for x in range(n_layers):
      md.add(layers.Dense(dense['input_units_{0}'.format(x+1)], activation=dense['activation_{0}'.format(x+1)]))
      md.add(layers.BatchNormalization())
    md.add(layers.Dense(2, activation=activation_end))
    md.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-06),
                  loss='mean_squared_error',metrics = ['accuracy'])
    return md

In [ ]:
bestModel = best_model(**best_hp)
bestModel.fit(X_train,y_train,
           epochs = 20,
           batch_size=200,
           shuffle=True,
           validation_data = (X_val,y_val))

Epoch 1/20
2272/2272 [==============================] - 19s 6ms/step - loss: 0.1091 - accuracy: 0.8638 - val_loss: 0.0875 - val_accuracy: 0.8902
Epoch 2/20
2272/2272 [==============================] - 14s 6ms/step - loss: 0.0926 - accuracy: 0.8818 - val_loss: 0.0825 - val_accuracy: 0.8930
Epoch 3/20
2272/2272 [==============================] - 14s 6ms/step - loss: 0.0887 - accuracy: 0.8853 - val_loss: 0.0806 - val_accuracy: 0.8946
Epoch 4/20
2272/2272 [==============================] - 15s 6ms/step - loss: 0.0861 - accuracy: 0.8883 - val_loss: 0.0788 - val_accuracy: 0.8972
Epoch 5/20
2272/2272 [==============================] - 14s 6ms/step - loss: 0.0843 - accuracy: 0.8900 - val_loss: 0.0779 - val_accuracy: 0.8967
Epoch 6/20
2272/2272 [==============================] - 14s 6ms/step - loss: 0.0827 - accuracy: 0.8917 - val_loss: 0.0775 - val_accuracy: 0.8959
Epoch 7/20
2272/2272 [==============================] - 14s 6ms/step - loss: 0.0815 - accuracy: 0.8935 - val_loss: 0.0767 - val_ac

In [ ]:
def encode_test(df,cat=cat_top):
  encoded_columns = encoder.transform(df[cat])
  encoded_column_names = encoder.get_feature_names_out()
  encoded_df = pd.DataFrame(encoded_columns, columns=encoded_column_names, index = df.index)
  feature_encoded = pd.concat([df.drop(columns=cat),encoded_df], axis=1)
  return stder(feature_encoded.astype(np.float64))

In [ ]:
def preprocess_test(df):
  df = fill_na(df)
  X_test_encoded = encode_test(df)
  return X_test_encoded

In [ ]:
pred = []
num.remove('target')
test = pd.read_csv('test.csv',chunksize = 40000)
for x in test:
  x.set_index('customer_ID', inplace = True)
  indx = x.index
  X_test = preprocess_test(x)
  prediction = pd.DataFrame(bestModel.predict(X_test)[:,1], columns=['prediction'])
  prediction.index = indx
  pred.append(prediction)
output = pd.concat(pred, axis=0)
output.to_csv('submission.csv')

145/145 [==============================] - 0s 2ms/step


In [ ]:
!kaggle competitions submit -c amex-default-prediction -f submission.csv -m "Message"

100% 67.1M/67.1M [00:02<00:00, 25.2MB/s]
Successfully submitted to American Express - Default Prediction